**Installing and Importing the Necessary Libraries**

In [ ]:
%%capture
# !pip install be-great

In [ ]:
%%capture
!pip install openpyxl

In [ ]:
# from be_great import GReaT
import pandas as pd
import torch
import xlrd
from openpyxl import load_workbook
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/Capgemini-Practicum

/content/drive/MyDrive/Capgemini-Practicum


In [ ]:
!rm -rf be_great/

In [ ]:
%pwd

'/content/drive/MyDrive/Capgemini-Practicum'

In [ ]:
!git clone https://github.com/kathrinse/be_great.git

Cloning into 'be_great'...
remote: Enumerating objects: 476, done.
remote: Counting objects: 100% (159/159), done.
remote: Compressing objects: 100% (110/110), done.
remote: Total 476 (delta 88), reused 90 (delta 46), pack-reused 317
Receiving objects: 100% (476/476), 4.25 MiB | 17.40 MiB/s, done.
Resolving deltas: 100% (224/224), done.


In [ ]:
%cd be_great

/content/drive/MyDrive/Capgemini-Practicum/be_great


In [ ]:
!pip install -r requirements.txt

In [ ]:
from be_great import GReaT

**Loading the DataSet**

In [ ]:
df_merged=pd.read_csv('Dataset/merged_med_pharmacy_2018.csv')

In [ ]:
df_merged=df_merged[['member_life_id', 'gender_code','claim_number','current_procedural_terminology','primary_diagnosis_code-icd10',
       'line_service_from_date', 'line_service_thru_date', 'billed_amount',
       'paid_amount','birth_date','ndc','drug_name','fill_date','paid_date', 'billed_amount_pharm', 'paid_amount_pharm']].reset_index(drop=True)

In [ ]:
df_merged.head()

In [ ]:
df_merged['fill_date'] = df_merged['fill_date'].str.slice(0, 10)

In [ ]:
df_merged['line_service_thru_date'] = df_merged['line_service_thru_date'].str.slice(0, 10)

In [ ]:
df_merged.head()

In [ ]:
df_merged=df_merged.sort_values(by=['member_life_id','claim_number', 'billed_amount'], ascending=[True,True, False]).drop_duplicates(subset='claim_number', keep='first').reset_index(drop=True)

In [ ]:
df_merged.shape

In [ ]:
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
sns.histplot(df_merged.groupby(by='member_life_id',as_index=False)['claim_number'].nunique()['claim_number'])
plt.xlim(0, 100)
plt.show()

In [ ]:
df_merged.groupby(by='member_life_id',as_index=False).agg({'claim_number':lambda x: (x.nunique(),x.count())})['claim_number'].apply(lambda x: 1 if x[0]!=x[1] else 0).sum()

In [ ]:
df_subset=df_merged.groupby(by='member_life_id',as_index=False)['claim_number'].nunique().rename(columns={'claim_number':'num_unique_claims'})
member_id_to_keep=list(df_subset[df_subset['num_unique_claims']>=3]['member_life_id'].unique())
df_merged_subset=df_merged[df_merged['member_life_id'].isin(member_id_to_keep)].reset_index(drop=True)

***Taking the patient id's with more than 3 visits and truncating the number for 3 latest visits at max.*** <br/>

This is done to ensure that we have a bigger sample to train

In [ ]:
df_merged_subset.columns

Index(['member_life_id', 'gender_code', 'claim_number',
       'current_procedural_terminology', 'primary_diagnosis_code-icd10',
       'line_service_from_date', 'line_service_thru_date', 'billed_amount',
       'paid_amount', 'birth_date', 'ndc', 'drug_name', 'fill_date',
       'paid_date', 'billed_amount_pharm', 'paid_amount_pharm'],
      dtype='object')

In [ ]:
df_merged_subset=df_merged_subset.sort_values(by=['member_life_id', 'line_service_from_date'], ascending=[True, False]).groupby('member_life_id', group_keys=False).apply(lambda g:g.head(3))

In [ ]:
df_merged_subset.groupby(by='member_life_id',as_index=False)['claim_number'].nunique()['claim_number'].unique()

array([3])

In [ ]:
assert (df_merged_subset.shape[0]/df_merged_subset.member_life_id.nunique())==3, "wrong transformation, please check!"

In [ ]:
# df_subset=df_merged.groupby(by='member_life_id',as_index=False)['claim_number'].nunique().rename(columns={'claim_number':'num_unique_claims'})
# member_id_to_keep=list(df_subset[df_subset['num_unique_claims']==1]['member_life_id'].unique())
# df_merged_subset=df_merged[df_merged['member_life_id'].isin(member_id_to_keep)].reset_index(drop=True)

In [ ]:
df_merged_subset.groupby(by='member_life_id',as_index=False)['claim_number'].count()['claim_number'].value_counts()

1    4816
Name: claim_number, dtype: int64

In [ ]:
df_test=df_merged[df_merged['member_life_id']==3205823]

In [ ]:
df_merged_subset['line_service_from_date']=pd.to_datetime(df_merged_subset['line_service_from_date'])
df_merged_subset['line_service_thru_date']=pd.to_datetime(df_merged_subset['line_service_thru_date'])
df_merged_subset['fill_date']=pd.to_datetime(df_merged_subset['fill_date'])
df_merged_subset['paid_date']=pd.to_datetime(df_merged_subset['paid_date'])

In [ ]:
df_merged_subset.dtypes

member_life_id                             int64
gender_code                               object
claim_number                              object
current_procedural_terminology            object
primary_diagnosis_code-icd10              object
line_service_from_date            datetime64[ns]
line_service_thru_date            datetime64[ns]
billed_amount                            float64
paid_amount                              float64
birth_date                                object
ndc                                        int64
drug_name                                 object
fill_date                         datetime64[ns]
paid_date                         datetime64[ns]
billed_amount_pharm                      float64
paid_amount_pharm                        float64
dtype: object

In [ ]:
# # Define a list of columns you want to apply the condition to
# columns_to_update = ['ndc','drug_name','fill_date','paid_date', 'billed_amount_pharm', 'paid_amount_pharm']

# # Apply np.where to the specified columns to set "not prescribed" if the date difference is less than 30 days
# for col in columns_to_update:
#   df_merged_subset[col]=np.where(df_merged_subset.apply(lambda x:(x.fill_date-x.line_service_from_date).days, axis=1)<30,'not prescribed',df_merged_subset[col])

In [ ]:
df_merged_subset=df_merged_subset.sort_values(by=['member_life_id', 'line_service_from_date'], ascending=[True, True]).reset_index(drop=True)

In [ ]:
df_merged_subset.head(6)

,member_life_id,gender_code,claim_number,current_procedural_terminology,primary_diagnosis_code-icd10,line_service_from_date,line_service_thru_date,billed_amount,paid_amount,birth_date,ndc,drug_name,fill_date,paid_date,billed_amount_pharm,paid_amount_pharm
0,169,F,83101B096100,99211,H903,2018-11-01,2018-11-01,40.00,0.00,1946-09-05,781552810,BUPROPN HCL TAB 150MG XL,2018-12-05,2018-12-05,49.88,35.16
1,169,F,833012276300,90662,Z23,2018-11-06,2018-11-06,38.22,38.22,1946-09-05,781552810,BUPROPN HCL TAB 150MG XL,2018-12-05,2018-12-05,49.88,35.16
2,169,F,834017645500,V5261,H903,2018-12-05,2018-12-05,5400.00,508.31,1946-09-05,781552810,BUPROPN HCL TAB 150MG XL,2018-12-05,2018-12-05,49.88,35.16
3,470,M,808817770600,T2023,F322,2018-01-12,2018-01-12,390.00,390.00,1985-12-31,781518110,LEVOTHYROXIN TAB 50MCG,2018-08-09,2018-08-19,-49.55,0.00
4,470,M,805812602600,T2022,R6889,2018-01-31,2018-01-31,395.00,395.00,1985-12-31,781518110,LEVOTHYROXIN TAB 50MCG,2018-08-09,2018-08-19,-49.55,0.00
5,470,M,813715164300,99214,F331,2018-05-16,2018-05-16,256.00,169.77,1985-12-31,781518110,LEVOTHYROXIN TAB 50MCG,2018-08-09,2018-08-19,-49.55,0.00


In [ ]:
df_merged_subset.columns

Index(['member_life_id', 'gender_code', 'claim_number',
       'current_procedural_terminology', 'primary_diagnosis_code-icd10',
       'line_service_from_date', 'line_service_thru_date', 'billed_amount',
       'paid_amount', 'birth_date', 'ndc', 'drug_name', 'fill_date',
       'paid_date', 'billed_amount_pharm', 'paid_amount_pharm'],
      dtype='object')

In [ ]:
df_merged_subset=df_merged_subset[['member_life_id','gender_code','birth_date',
       'current_procedural_terminology', 'primary_diagnosis_code-icd10',
       'line_service_from_date', 'line_service_thru_date', 'billed_amount',
       'paid_amount','ndc', 'drug_name', 'fill_date',
       'paid_date', 'billed_amount_pharm', 'paid_amount_pharm']].rename(columns={'member_life_id':'patient_id','current_procedural_terminology':'cpt_code','primary_diagnosis_code-icd10':'primary_diagnosis_code',\
                                       'line_service_from_date':'visit_start_date','line_service_thru_date':'visit_end_date'})

In [ ]:
df_merged_subset.columns

Index(['patient_id', 'gender_code', 'birth_date', 'cpt_code',
       'primary_diagnosis_code', 'visit_start_date', 'visit_end_date',
       'billed_amount', 'paid_amount', 'ndc', 'drug_name', 'fill_date',
       'paid_date', 'billed_amount_pharm', 'paid_amount_pharm'],
      dtype='object')

In [ ]:
df_merged_subset['visit_start_date']=df_merged_subset['visit_start_date'].astype(str)
df_merged_subset['visit_end_date']=df_merged_subset['visit_end_date'].astype(str)
df_merged_subset['fill_date']=df_merged_subset['fill_date'].astype(str)
df_merged_subset['paid_date']=df_merged_subset['paid_date'].astype(str)

In [ ]:
# col_remove_list=['patient_id', 'gender_code', 'birth_date']
# df_merged_subset.groupby(col_remove_list,as_index=False)['fill_date'].count()

,patient_id,gender_code,birth_date,fill_date
0,470,M,1985-12-31,3
1,870,F,1968-03-28,3
2,4118,F,1990-10-08,3
3,8832,F,1991-02-22,3
4,13985,M,1962-10-17,3
...,...,...,...,...
3623,35816711,M,1961-03-11,3
3624,35819285,M,1980-06-20,3
3625,35853485,M,1975-08-01,3
3626,36076323,M,1975-01-04,3


In [ ]:
df_merged_subset.shape

(91071, 15)

In [ ]:
df_merged_subset.groupby(['patient_id', 'gender_code', 'cpt_code',
       'primary_diagnosis_code', 'visit_start_date', 'visit_end_date',
       'billed_amount', 'paid_amount'],as_index=False)['fill_date'].nunique()['fill_date'].value_counts()

1    91071
Name: fill_date, dtype: int64

**Function to transform multiple visits into a unified prompt**

In [ ]:
import pandas as pd

# Assuming we have a dataframe 'df' with the necessary columns.
# The function to be applied across the dataframe would look something like this:
# col_remove_list=['patient_id', 'gender_code', 'birth_date','ndc', 'drug_name', 'fill_date',
#        'paid_date', 'billed_amount_pharm', 'paid_amount_pharm']

col_remove_list=['patient_id', 'gender_code', 'birth_date']
def format_visit_information(row):
    # Extract the column names except 'patient_id' and 'gender_code'
    global col_remove_list
    column_names = row.index.tolist()
    column_names=filter(lambda i: i not in col_remove_list, column_names)

    # Build the visit information string with column names and values
    visit_info = ", ".join(f"{col} is {row[col]}" for col in column_names)
    return f"{visit_info}"

# Modified function to concatenate visit information with each visit on a new line
def concatenate_visits(group):
    visit_details = group.apply(format_visit_information, axis=1).tolist()
    return "\n".join(f"<visit_{i+1}> : {details}" for i, details in enumerate(visit_details))

# Group by 'patient_id' and 'gender_code' and apply the concatenation function
# grouped_info = df.groupby(['patient_id', 'gender_code']).apply(concatenate_visits).reset_index(name='visit_info')

# The 'grouped_info' DataFrame will now have a summary string for each patient.

In [ ]:
grouped_info = df_merged_subset.groupby(col_remove_list).apply(concatenate_visits).reset_index(name='visit_info')

grouped_info

,patient_id,gender_code,birth_date,visit_info
0,169,F,1946-09-05,"<visit_1> : cpt_code is 99211, primary_diagnos..."
1,470,M,1985-12-31,"<visit_1> : cpt_code is T2023, primary_diagnos..."
2,747,F,1963-04-27,"<visit_1> : cpt_code is 99203, primary_diagnos..."
3,870,F,1968-03-28,"<visit_1> : cpt_code is 86592, primary_diagnos..."
4,1158,M,1960-05-27,"<visit_1> : cpt_code is 93351, primary_diagnos..."
...,...,...,...,...
30368,36113738,F,1973-06-20,"<visit_1> : cpt_code is 99203, primary_diagnos..."
30369,38155835,M,1989-06-06,"<visit_1> : cpt_code is 76700, primary_diagnos..."
30370,38156140,F,1991-02-28,"<visit_1> : cpt_code is 99214, primary_diagnos..."
30371,38503401,F,1959-04-03,"<visit_1> : cpt_code is 80061, primary_diagnos..."


In [ ]:
print(grouped_info.iloc[0]['visit_info'])

<visit_1> : cpt_code is 99211, primary_diagnosis_code is H903, visit_start_date is 2018-11-01, visit_end_date is 2018-11-01, billed_amount is 40.0, paid_amount is 0.0, ndc is 781552810, drug_name is BUPROPN HCL  TAB 150MG XL, fill_date is 2018-12-05, paid_date is 2018-12-05, billed_amount_pharm is 49.88, paid_amount_pharm is 35.16
<visit_2> : cpt_code is 90662, primary_diagnosis_code is Z23, visit_start_date is 2018-11-06, visit_end_date is 2018-11-06, billed_amount is 38.22, paid_amount is 38.22, ndc is 781552810, drug_name is BUPROPN HCL  TAB 150MG XL, fill_date is 2018-12-05, paid_date is 2018-12-05, billed_amount_pharm is 49.88, paid_amount_pharm is 35.16
<visit_3> : cpt_code is V5261, primary_diagnosis_code is H903, visit_start_date is 2018-12-05, visit_end_date is 2018-12-05, billed_amount is 5400.0, paid_amount is 508.31, ndc is 781552810, drug_name is BUPROPN HCL  TAB 150MG XL, fill_date is 2018-12-05, paid_date is 2018-12-05, billed_amount_pharm is 49.88, paid_amount_pharm is 

In [ ]:
grouped_info.to_csv('Dataset/patient_merged_visit_info_truncated_3_visits.csv')

**Transforming each row into extra columns - Serves as Input to LLM**

In [ ]:
import pandas as pd
import re
# Function to extract and transform the data
def extract_and_transform(row):
    # Find all the visits
    visits = re.findall(r"<(visit_\d+)> : (.+?)\s*(?=<visit_\d+>|$)", row)

    # Dictionary to hold the extracted data
    extracted_data = {}

    for visit in visits:
        visit_name, visit_data = visit
        # Find all the key-value pairs in the visit
        key_values = re.findall(r"(\w+) is ([\w\d.-]+)", visit_data)
        for key, value in key_values:
            # Construct the new column name
            column_name = f"{visit_name}_{key}"
            extracted_data[column_name] = value

    return extracted_data

# Apply the function to each row and create a new DataFrame

extracted_rows = grouped_info['visit_info'].apply(extract_and_transform)
df_merged_transformed = pd.concat([grouped_info[col_remove_list],pd.DataFrame(extracted_rows.tolist())],axis=1)

df_merged_transformed.head()

,patient_id,gender_code,birth_date,visit_1_cpt_code,visit_1_primary_diagnosis_code,visit_1_visit_start_date,visit_1_visit_end_date,visit_1_billed_amount,visit_1_paid_amount,visit_1_ndc,...,visit_3_visit_start_date,visit_3_visit_end_date,visit_3_billed_amount,visit_3_paid_amount,visit_3_ndc,visit_3_drug_name,visit_3_fill_date,visit_3_paid_date,visit_3_billed_amount_pharm,visit_3_paid_amount_pharm
0,169,F,1946-09-05,99211,H903,2018-11-01,2018-11-01,40.0,0.0,781552810,...,2018-12-05,2018-12-05,5400.0,508.31,781552810,BUPROPN,2018-12-05,2018-12-05,49.88,35.16
1,470,M,1985-12-31,T2023,F322,2018-01-12,2018-01-12,390.0,390.0,781518110,...,2018-05-16,2018-05-16,256.0,169.77,781518110,LEVOTHYROXIN,2018-08-09,2018-08-19,-49.55,0.0
2,747,F,1963-04-27,99203,K582,2018-12-19,2018-12-19,149.0,0.0,43199001101,...,2018-12-31,2018-12-31,234.0,0.0,24658031205,DOXYCYCL,2018-12-31,2018-12-31,371.26,18.71
3,870,F,1968-03-28,86592,Z01419,2018-06-28,2018-06-28,15.0,4.23,69452015120,...,2018-07-25,2018-07-25,375.72,0.0,69452015120,VITAMIN,2018-08-31,2018-09-14,-3.94,-2.1
4,1158,M,1960-05-27,93351,I208,2018-09-21,2018-09-21,725.0,0.0,69452010532,...,2018-11-05,2018-11-05,19.99,18.2,69452010532,MONTELUKAST,2018-12-11,2018-12-11,146.31,5.71


In [ ]:
df_merged_transformed['patient_id']=df_merged_transformed['patient_id'].map({k:'p_' + str(v) for k,v in dict(zip(df_merged_transformed['patient_id'],range(df_merged_transformed.shape[0]))).items()})

In [ ]:
df_merged_transformed.head(),

,patient_id,gender_code,birth_date,visit_1_cpt_code,visit_1_primary_diagnosis_code,visit_1_visit_start_date,visit_1_visit_end_date,visit_1_billed_amount,visit_1_paid_amount,visit_1_ndc,...,visit_3_visit_start_date,visit_3_visit_end_date,visit_3_billed_amount,visit_3_paid_amount,visit_3_ndc,visit_3_drug_name,visit_3_fill_date,visit_3_paid_date,visit_3_billed_amount_pharm,visit_3_paid_amount_pharm
0,p_0,F,1946-09-05,99211,H903,2018-11-01,2018-11-01,40.0,0.0,781552810,...,2018-12-05,2018-12-05,5400.0,508.31,781552810,BUPROPN,2018-12-05,2018-12-05,49.88,35.16
1,p_1,M,1985-12-31,T2023,F322,2018-01-12,2018-01-12,390.0,390.0,781518110,...,2018-05-16,2018-05-16,256.0,169.77,781518110,LEVOTHYROXIN,2018-08-09,2018-08-19,-49.55,0.0
2,p_2,F,1963-04-27,99203,K582,2018-12-19,2018-12-19,149.0,0.0,43199001101,...,2018-12-31,2018-12-31,234.0,0.0,24658031205,DOXYCYCL,2018-12-31,2018-12-31,371.26,18.71
3,p_3,F,1968-03-28,86592,Z01419,2018-06-28,2018-06-28,15.0,4.23,69452015120,...,2018-07-25,2018-07-25,375.72,0.0,69452015120,VITAMIN,2018-08-31,2018-09-14,-3.94,-2.1
4,p_4,M,1960-05-27,93351,I208,2018-09-21,2018-09-21,725.0,0.0,69452010532,...,2018-11-05,2018-11-05,19.99,18.2,69452010532,MONTELUKAST,2018-12-11,2018-12-11,146.31,5.71


In [ ]:
df_merged_transformed.shape

(30373, 39)

In [ ]:
df_merged_transformed.to_csv('/content/drive/MyDrive/Capgemini-Practicum/Dataset/df_merged_truncated_3_visits.csv',index=False)

In [ ]:
df_merged_transformed=pd.read_csv('/content/drive/MyDrive/Capgemini-Practicum/Dataset/df_merged_truncated_3_visits.csv',index_col=False)
df_merged_transformed.head()

,patient_id,gender_code,birth_date,visit_1_cpt_code,visit_1_primary_diagnosis_code,visit_1_visit_start_date,visit_1_visit_end_date,visit_1_billed_amount,visit_1_paid_amount,visit_1_ndc,...,visit_3_visit_start_date,visit_3_visit_end_date,visit_3_billed_amount,visit_3_paid_amount,visit_3_ndc,visit_3_drug_name,visit_3_fill_date,visit_3_paid_date,visit_3_billed_amount_pharm,visit_3_paid_amount_pharm
0,p_0,F,1946-09-05,99211,H903,2018-11-01,2018-11-01,40.0,0.00,781552810,...,2018-12-05,2018-12-05,5400.00,508.31,7.815528e+08,BUPROPN,2018-12-05,2018-12-05,49.88,35.16
1,p_1,M,1985-12-31,T2023,F322,2018-01-12,2018-01-12,390.0,390.00,781518110,...,2018-05-16,2018-05-16,256.00,169.77,7.815181e+08,LEVOTHYROXIN,2018-08-09,2018-08-19,-49.55,0.00
2,p_2,F,1963-04-27,99203,K582,2018-12-19,2018-12-19,149.0,0.00,43199001101,...,2018-12-31,2018-12-31,234.00,0.00,2.465803e+10,DOXYCYCL,2018-12-31,2018-12-31,371.26,18.71
3,p_3,F,1968-03-28,86592,Z01419,2018-06-28,2018-06-28,15.0,4.23,69452015120,...,2018-07-25,2018-07-25,375.72,0.00,6.945202e+10,VITAMIN,2018-08-31,2018-09-14,-3.94,-2.10
4,p_4,M,1960-05-27,93351,I208,2018-09-21,2018-09-21,725.0,0.00,69452010532,...,2018-11-05,2018-11-05,19.99,18.20,6.945201e+10,MONTELUKAST,2018-12-11,2018-12-11,146.31,5.71


In [ ]:
df_merged_transformed.shape

(30373, 39)

**Concatenating the Data for patients with 1,2 and 3 visits (Patients with more than 3 visits have been truncated to pick the 3 most recent visits)**

In [ ]:
import pandas as pd
import re

def transform_dataframe(df):
    # Function to rename columns using regex
    def rename_columns(df_visit, visit_number):
        new_columns = {}
        for col in df_visit.columns:
            match = re.match(f'visit_{visit_number}_(.*)', col)
            if match:
                new_columns[col] = match.group(1)
        return df_visit.rename(columns=new_columns)

    # Extracting visit numbers
    visit_numbers = set(re.match(r'visit_(\d+)_', col).group(1) for col in df.columns if re.match(r'visit_(\d+)_', col))
    sorted_visit_numbers = sorted(visit_numbers)

    # Creating separate dataframes for each visit
    dfs = []
    for i in sorted_visit_numbers:
        cols_to_keep = ['patient_id', 'gender_code', 'birth_date']
        cols_for_visit = [col for col in df.columns if f'visit_{i}_' in col]
        df_visit = df[cols_to_keep + cols_for_visit].copy()
        df_visit = rename_columns(df_visit, i)
        dfs.append(df_visit)

    # Concatenating all visits into one DataFrame
    return pd.concat(dfs, ignore_index=True)

In [ ]:
df_merged_transformed.columns

Index(['patient_id', 'gender_code', 'birth_date', 'visit_1_cpt_code',
       'visit_1_primary_diagnosis_code', 'visit_1_visit_start_date',
       'visit_1_visit_end_date', 'visit_1_billed_amount',
       'visit_1_paid_amount', 'visit_1_ndc', 'visit_1_drug_name',
       'visit_1_fill_date', 'visit_1_paid_date', 'visit_1_billed_amount_pharm',
       'visit_1_paid_amount_pharm', 'visit_2_cpt_code',
       'visit_2_primary_diagnosis_code', 'visit_2_visit_start_date',
       'visit_2_visit_end_date', 'visit_2_billed_amount',
       'visit_2_paid_amount', 'visit_2_ndc', 'visit_2_drug_name',
       'visit_2_fill_date', 'visit_2_paid_date', 'visit_2_billed_amount_pharm',
       'visit_2_paid_amount_pharm', 'visit_3_cpt_code',
       'visit_3_primary_diagnosis_code', 'visit_3_visit_start_date',
       'visit_3_visit_end_date', 'visit_3_billed_amount',
       'visit_3_paid_amount', 'visit_3_ndc', 'visit_3_drug_name',
       'visit_3_fill_date', 'visit_3_paid_date', 'visit_3_billed_amount_pharm',

In [ ]:
df_merged_transformed['visit_1_ndc']=df_merged_transformed['visit_1_ndc'].astype(str)
df_merged_transformed['visit_2_ndc']=df_merged_transformed['visit_2_ndc'].astype(str)
df_merged_transformed['visit_3_ndc']=df_merged_transformed['visit_3_ndc'].astype(str)

In [ ]:
df_merged_transformed_subset=df_merged_transformed.sample(15000,random_state=42).reset_index(drop=True)

In [ ]:
overall_dataframe=transform_dataframe(df_merged_transformed_subset)

In [ ]:
df_merged_transformed_subset

,patient_id,gender_code,birth_date,visit_1_cpt_code,visit_1_primary_diagnosis_code,visit_1_visit_start_date,visit_1_visit_end_date,visit_1_billed_amount,visit_1_paid_amount,visit_1_ndc,...,visit_3_visit_start_date,visit_3_visit_end_date,visit_3_billed_amount,visit_3_paid_amount,visit_3_ndc,visit_3_drug_name,visit_3_fill_date,visit_3_paid_date,visit_3_billed_amount_pharm,visit_3_paid_amount_pharm
0,p_7743,M,2002-07-18,99214,J020,2018-01-27,2018-01-27,175.00,62.20,66685100200,...,2018-08-19,2018-08-19,297.00,66.50,66685100200.0,AMOX,2018-08-19,2018-08-19,81.19,0.00
1,p_14187,F,1966-06-22,99214,E039,2018-11-16,2018-11-16,145.00,52.59,527134310,...,2018-12-15,2018-12-15,51.00,2.83,527134310.0,LEVOTHYROXIN,2018-12-15,2018-12-15,16.44,0.00
2,p_22892,F,2015-06-02,99392,Z00129,2018-07-26,2018-07-26,183.00,175.19,60758090810,...,2018-12-03,2018-12-03,151.00,63.31,60758090810.0,POLYMYXIN,2018-12-03,2018-12-03,27.23,0.00
3,p_2034,F,1955-02-08,93000,I509,2018-11-29,2018-11-29,54.00,48.27,52817033200,...,2018-12-24,2018-12-24,220.08,47.12,52817033200.0,CYCLOBENZAPR,2018-12-28,2018-12-28,98.23,0.00
4,p_16434,M,1970-05-27,99204,E038,2018-07-03,2018-07-03,340.00,101.74,42794001812,...,2018-08-06,2018-08-06,190.00,72.29,74929690.0,SYNTHROID,2018-09-30,2018-09-30,285.70,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,p_20514,M,1974-11-04,73590,S8992XA,2018-11-28,2018-11-28,63.00,0.00,42858010201,...,2018-12-10,2018-12-10,150.00,0.00,53746010901.0,HYDROCO,2018-12-10,2018-12-10,53.78,9.34
14996,p_8762,F,1951-07-05,99214,R938,2018-02-27,2018-02-27,180.00,72.48,59762500701,...,2018-03-26,2018-03-26,223.00,67.77,59762500701.0,MISOPROSTOL,2018-04-23,2018-04-23,3.32,0.00
14997,p_5761,F,1975-05-11,87086,R300,2018-11-08,2018-11-08,60.75,0.00,61958220101,...,2018-12-12,2018-12-12,160.00,55.20,61958220101.0,EPCLUSA,2018-12-17,2018-12-26,-52337.00,0.00
14998,p_12441,M,1969-05-03,81003,R55,2018-09-30,2018-09-30,42.00,0.00,68180016013,...,2018-10-04,2018-10-04,304.00,0.00,68180016013.0,AZITHROMYCIN,2018-10-04,2018-10-04,49.20,4.49


In [ ]:
overall_dataframe.to_csv('/content/drive/MyDrive/Capgemini-Practicum/Dataset/df_untransformed_45k.csv',index=False)

In [ ]:
import torch

if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU is available")
else:
    device = torch.device("cpu")
    print("GPU is not available; using CPU")

torch.cuda.empty_cache()

GPU is available


In [ ]:
from be_great import GReaT

**Loading the GReaT model for finetuning on new transformed data**

In [ ]:
gpt2_base_model = GReaT(llm='gpt2', batch_size=4, epochs=3,
              experiment_dir="/content/drive/MyDrive/Capgemini-Practicum/ModelDataCheckpoints/trainer_medical_transformed_truncated_3_visits",learning_rate=2e-5,save_steps=500)

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
df_merged_transformed_subset.shape

(15000, 39)

In [ ]:
gpt2_base_model.fit(df_merged_transformed_subset)

You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
500,0.449100
1000,0.368500
1500,0.352000
2000,0.341700
2500,0.332700
3000,0.324000
3500,0.320400
4000,0.316100
4500,0.311500
5000,0.307500


Saving the model checkpoint

In [ ]:
torch.save(gpt2_base_model, '/content/drive/MyDrive/Capgemini-Practicum/ModelDataCheckpoints/gpt2_model_truncated_3_15k.pth')

In [ ]:
gpt2_base_model=torch.load('/content/drive/MyDrive/Capgemini-Practicum/ModelDataCheckpoints/gpt2_model_truncated_3_15k.pth')

**Code to Generate the Synthetic Data from the finetuned model**

In [ ]:
synthetic_data_row_transformed = gpt2_base_model.sample(n_samples=500,max_length=1024,k=200,device="cpu")

  0%|          | 0/500 [00:00<?, ?it/s]

In [ ]:
synthetic_merged_data_baseline.head(10)

,patient_id,gender_code,birth_date,visit_1_cpt_code,visit_1_primary_diagnosis_code,visit_1_visit_start_date,visit_1_visit_end_date,visit_1_billed_amount,visit_1_paid_amount,visit_1_ndc,visit_1_drug_name,visit_1_fill_date,visit_1_paid_date,visit_1_billed_amount_pharm,visit_1_paid_amount_pharm
0,p_2535,M,1992-05-24,99213,J029,2018-06-26,2018-06-26,0.0,0.0,5.941750e+09,CLINDAMYCYCL,2018-06-26,2018-06-26,-26.42,0.00
1,p_4383,M,2010-06-27,99213,R04,2018-01-20,2018-01-20,99.0,0.0,5.186803e+10,AMPHETAMINE,2018-01-20,2018-01-20,0.00,0.00
2,p_3665,F,1968-11-22,99214,D939,2018-04-29,2018-04-29,49.0,0.0,5.400501e+07,CEFDINIR,2018-04-29,2018-04-29,0.00,0.00
3,p_2664,M,1993-08-06,99383,J111,2018-05-29,2018-05-29,120.0,0.0,1.671402e+10,IBUPROFEN,2018-06-27,2018-06-27,0.00,0.00
4,p_1717,M,1989-09-08,99214,Z0000,2018-01-25,2018-01-25,170.0,0.0,7.043206e+08,PREDNISONE,2018-01-25,2018-01-25,37.54,0.00
5,p_1046,F,1994-05-14,99213,Z01411,2018-05-03,2018-05-03,15.0,0.0,6.816007e+10,AMOXICILLIN,2018-05-03,2018-05-03,10.76,0.00
6,p_4341,F,1985-04-09,99213,R300,2018-04-23,2018-04-23,206.0,0.0,6.060451e+07,ZONE,2018-04-24,2018-04-24,-35.60,0.00
7,p_2388,M,1970-11-27,99213,T0302,2018-02-05,2018-02-05,120.0,0.0,6.846202e+10,AMOXICILLIN,2018-04-08,2018-04-08,0.00,4.47
8,p_3126,M,1965-02-08,99203,H6692,2018-04-26,2018-04-26,150.0,0.0,5.034000e+09,PROAIR,2018-05-25,2018-04-26,10.58,0.00
9,p_3495,M,1987-07-30,99213,J029,2018-01-21,2018-01-21,35.0,0.0,6.818002e+10,OMEPRAZOLE,2018-01-21,2018-01-21,0.00,0.00


In [ ]:
synthetic_merged_data_baseline.to_csv('/content/drive/MyDrive/Capgemini-Practicum/Dataset/gpt2_model_baseline_merged_df_v2.csv',index=False)

In [ ]:
diamonds.select_dtypes(include=np.number).head()